In [2]:
from sklearn.feature_extraction.text import CountVectorizer
import preprocessor
import gensim
import json
import preprocessing
import re
import string
import pandas as pd
import functions as f
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
# gensim
from gensim import corpora, models, similarities, matutils
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
corpus = f.read_json('/home/nick/Downloads/reddit_comments/RC_2007-12')
corpus

,author,id,subreddit,author_flair_css_class,downs,body,edited,retrieved_on,subreddit_id,distinguished,...,created_utc,name,score_hidden,parent_id,controversiality,score,author_flair_text,link_id,archived,ups
0,[deleted],c02khd5,politics,None,0,The fact remains that Chavez does not understa...,False,1427421031,t5_2cneq,None,...,1196467200,t1_c02khd5,False,t1_c02kdcg,0,1,None,t3_61tiq,True,1
1,[deleted],c02khd6,politics,None,0,[deleted],False,1427421031,t5_2cneq,None,...,1196467207,t1_c02khd6,False,t3_61u0v,0,0,None,t3_61u0v,True,0
2,ScrewDriver,c02khd7,reddit.com,None,0,Why we admired him most` never be afraid risk ...,False,1427421031,t5_6,None,...,1196467207,t1_c02khd7,False,t3_61utp,0,1,None,t3_61utp,True,1
3,[deleted],c02khd8,reddit.com,None,0,"No, Mom! NOOOOOOOOOOOOOOOOOOOOOOOO!",False,1427421031,t5_6,None,...,1196467208,t1_c02khd8,False,t3_61usz,0,74,None,t3_61usz,True,74
4,Deacon,c02khd9,reddit.com,None,0,"Great, so he only had sex with *one* nine-year...",False,1427421031,t5_6,None,...,1196467224,t1_c02khd9,False,t1_c02khbz,0,54,None,t3_61ukl,True,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363385,rowd149,c02s9s1,reddit.com,None,0,So... so Dinotopia *could* exist? :D,False,1427417259,t5_6,None,...,1199145579,t1_c02s9s1,False,t1_c02s8g5,0,5,None,t3_6482i,True,5
363386,mlietzen,c02s9s2,politics,None,0,Don't you wish Americans cared this much about...,False,1427417259,t5_2cneq,None,...,1199145582,t1_c02s9s2,False,t3_648hg,0,1,None,t3_648hg,True,1
363387,bbqribs,c02s9s3,reddit.com,None,0,The illegals around here said in interviews th...,False,1427417259,t5_6,None,...,1199145583,t1_c02s9s3,False,t1_c02s8lb,0,0,None,t3_647mf,True,0
363388,joyork,c02s9s4,reddit.com,None,0,For the first time ever on NYE I'm alone and s...,False,1427417259,t5_6,None,...,1199145594,t1_c02s9s4,False,t3_648on,0,1,None,t3_648on,True,1


In [4]:
deleted_rows = corpus[corpus['body'] == '[deleted]'].index
corpus.drop(deleted_rows, inplace=True)

In [7]:
comments = corpus.iloc[:, 5]
comments = [str(comment) for comment in comments]

In [8]:
comments = [comment.lower() for comment in comments]
comments = [x.split('http', 1)[0] for x in comments]
comments = [x.split('pic.',1)[0] for x in comments]
comments = [re.sub('[%s]' % re.escape(string.punctuation), '', text)  for text in comments]
comments

['the fact remains that chavez does not understand economics hes imposed price controls',
 'deleted',
 'why we admired him most never be afraid risk it all believe in yourself',
 'no mom noooooooooooooooooooooooo',
 'great so he only had sex with one nineyearold girl for a moment i was worried',
 'deleted',
 'if reddit or anybody in the conde nast family uses adwords or adsense than they cant advocate clicking on the ads either to drive up another advertisers bill adwords or to make some web publishher money adsense\n\nif reddits users take it upon themselves to click on somebodys google ads with the intention of making things expensive for them then they are exposing reddit to civil liability which is covered under the user agreement\n\nthe individuals participating in this arent liable for anything but reddit would be if they allowed it to happen',
 'its because wiis are sold out and the price drop and im willing to bet a lot of the people buying up the ps3s now are already wii owner

In [9]:
# Create a CountVectorizer for parsing/counting words
count_vectorizer = CountVectorizer(ngram_range=(1, 2),  
                                   stop_words='english', 
                                   strip_accents='unicode', 
                                   token_pattern="\\b[a-z][a-z]+\\b", 
                                   lowercase=True,
                                  )

comms = count_vectorizer.fit(comments)

In [10]:
comms

CountVectorizer(ngram_range=(1, 2), stop_words='english',
                strip_accents='unicode', token_pattern='\\b[a-z][a-z]+\\b')

In [11]:
# Create the term-document matrix
# Transpose it so the terms are the rows
doc_word = count_vectorizer.transform(comments).transpose()

In [12]:
doc_word[1]

<1x363390 sparse matrix of type '<class 'numpy.int64'>'
	with 1 stored elements in Compressed Sparse Column format>

In [13]:
df = pd.DataFrame(doc_word.toarray(), count_vectorizer.get_feature_names())
df = df.loc[df.index.drop_duplicates()]

MemoryError: Unable to allocate 7.46 TiB for an array with shape (2821736, 363390) and data type int64

In [14]:
doc_word.shape

(2821736, 363390)

In [15]:
# Convert sparse matrix of counts to a gensim corpus
corpus = matutils.Sparse2Corpus(doc_word)

In [16]:
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())
len(id2word)

2821736

In [17]:
# Create lda model (equivalent to "fit" in sklearn)
lda = models.LdaMulticore(corpus=corpus, num_topics=2, id2word=id2word, passes=5)

2020-08-09 10:42:17,968 : INFO : using symmetric alpha at 0.5
2020-08-09 10:42:17,969 : INFO : using symmetric eta at 0.5
2020-08-09 10:42:18,178 : INFO : using serial LDA version on this node
2020-08-09 10:42:18,501 : INFO : running online LDA training, 2 topics, 5 passes over the supplied corpus of 363390 documents, updating every 30000 documents, evaluating every ~300000 documents, iterating 50x with a convergence threshold of 0.001000
2020-08-09 10:42:18,501 : INFO : training LDA model using 15 processes
2020-08-09 10:42:18,700 : INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #2000/363390, outstanding queue size 1
2020-08-09 10:42:19,580 : INFO : PROGRESS: pass 0, dispatched chunk #1 = documents up to #4000/363390, outstanding queue size 2
2020-08-09 10:42:19,603 : INFO : PROGRESS: pass 0, dispatched chunk #2 = documents up to #6000/363390, outstanding queue size 3
2020-08-09 10:42:19,622 : INFO : PROGRESS: pass 0, dispatched chunk #3 = documents up to #8000/363390,

2020-08-09 10:42:45,227 : INFO : PROGRESS: pass 0, dispatched chunk #55 = documents up to #112000/363390, outstanding queue size 32
2020-08-09 10:42:46,265 : INFO : PROGRESS: pass 0, dispatched chunk #56 = documents up to #114000/363390, outstanding queue size 32
2020-08-09 10:42:47,228 : INFO : PROGRESS: pass 0, dispatched chunk #57 = documents up to #116000/363390, outstanding queue size 32
2020-08-09 10:42:48,180 : INFO : PROGRESS: pass 0, dispatched chunk #58 = documents up to #118000/363390, outstanding queue size 32
2020-08-09 10:42:49,124 : INFO : PROGRESS: pass 0, dispatched chunk #59 = documents up to #120000/363390, outstanding queue size 32
2020-08-09 10:42:50,135 : INFO : PROGRESS: pass 0, dispatched chunk #60 = documents up to #122000/363390, outstanding queue size 32
2020-08-09 10:42:51,280 : INFO : merging changes from 30000 documents into a model of 363390 documents
2020-08-09 10:42:52,387 : INFO : topic #0 (0.500): 0.005*"people" + 0.003*"just" + 0.002*"like" + 0.002*"

2020-08-09 10:43:34,114 : INFO : PROGRESS: pass 0, dispatched chunk #104 = documents up to #210000/363390, outstanding queue size 32
2020-08-09 10:43:35,075 : INFO : PROGRESS: pass 0, dispatched chunk #105 = documents up to #212000/363390, outstanding queue size 32
2020-08-09 10:43:36,218 : INFO : merging changes from 30000 documents into a model of 363390 documents
2020-08-09 10:43:37,329 : INFO : topic #0 (0.500): 0.005*"people" + 0.002*"just" + 0.002*"like" + 0.002*"think" + 0.002*"dont" + 0.002*"gt" + 0.001*"im" + 0.001*"know" + 0.001*"deleted" + 0.001*"right"
2020-08-09 10:43:37,351 : INFO : topic #1 (0.500): 0.006*"deleted" + 0.005*"like" + 0.005*"dont" + 0.004*"just" + 0.004*"people" + 0.003*"think" + 0.003*"im" + 0.002*"know" + 0.002*"time" + 0.002*"thats"
2020-08-09 10:43:37,358 : INFO : topic diff=0.109492, rho=0.128037
2020-08-09 10:43:38,279 : INFO : PROGRESS: pass 0, dispatched chunk #106 = documents up to #214000/363390, outstanding queue size 32
2020-08-09 10:43:40,288 :

2020-08-09 10:44:22,123 : INFO : topic #1 (0.500): 0.006*"deleted" + 0.005*"like" + 0.005*"dont" + 0.005*"just" + 0.004*"people" + 0.004*"think" + 0.003*"im" + 0.002*"know" + 0.002*"thats" + 0.002*"time"
2020-08-09 10:44:22,130 : INFO : topic diff=0.083829, rho=0.097129
2020-08-09 10:44:23,003 : INFO : PROGRESS: pass 0, dispatched chunk #151 = documents up to #304000/363390, outstanding queue size 32
2020-08-09 10:44:24,088 : INFO : PROGRESS: pass 0, dispatched chunk #152 = documents up to #306000/363390, outstanding queue size 29
2020-08-09 10:44:24,097 : INFO : PROGRESS: pass 0, dispatched chunk #153 = documents up to #308000/363390, outstanding queue size 30
2020-08-09 10:44:24,108 : INFO : PROGRESS: pass 0, dispatched chunk #154 = documents up to #310000/363390, outstanding queue size 31
2020-08-09 10:44:24,964 : INFO : PROGRESS: pass 0, dispatched chunk #155 = documents up to #312000/363390, outstanding queue size 32
2020-08-09 10:44:25,921 : INFO : PROGRESS: pass 0, dispatched ch

2020-08-09 10:45:23,300 : INFO : PROGRESS: pass 1, dispatched chunk #10 = documents up to #22000/363390, outstanding queue size 11
2020-08-09 10:45:24,121 : INFO : PROGRESS: pass 1, dispatched chunk #11 = documents up to #24000/363390, outstanding queue size 12
2020-08-09 10:45:24,233 : INFO : PROGRESS: pass 1, dispatched chunk #12 = documents up to #26000/363390, outstanding queue size 12
2020-08-09 10:45:24,241 : INFO : PROGRESS: pass 1, dispatched chunk #13 = documents up to #28000/363390, outstanding queue size 13
2020-08-09 10:45:25,134 : INFO : PROGRESS: pass 1, dispatched chunk #14 = documents up to #30000/363390, outstanding queue size 14
2020-08-09 10:45:26,092 : INFO : PROGRESS: pass 1, dispatched chunk #15 = documents up to #32000/363390, outstanding queue size 14
2020-08-09 10:45:26,205 : INFO : PROGRESS: pass 1, dispatched chunk #16 = documents up to #34000/363390, outstanding queue size 14
2020-08-09 10:45:26,214 : INFO : PROGRESS: pass 1, dispatched chunk #17 = documents

...

2020-08-09 10:53:41,261 : INFO : PROGRESS: pass 3, dispatched chunk #173 = documents up to #348000/363390, outstanding queue size 31
2020-08-09 10:53:42,216 : INFO : PROGRESS: pass 3, dispatched chunk #174 = documents up to #350000/363390, outstanding queue size 31
2020-08-09 10:53:42,389 : INFO : PROGRESS: pass 3, dispatched chunk #175 = documents up to #352000/363390, outstanding queue size 31
2020-08-09 10:53:44,271 : INFO : PROGRESS: pass 3, dispatched chunk #176 = documents up to #354000/363390, outstanding queue size 31
2020-08-09 10:53:44,380 : INFO : PROGRESS: pass 3, dispatched chunk #177 = documents up to #356000/363390, outstanding queue size 31
2020-08-09 10:53:45,366 : INFO : PROGRESS: pass 3, dispatched chunk #178 = documents up to #358000/363390, outstanding queue size 31
2020-08-09 10:53:47,158 : INFO : PROGRESS: pass 3, dispatched chunk #179 = documents up to #360000/363390, outstanding queue size 31
2020-08-09 10:53:48,098 : INFO : PROGRESS: pass 3, dispatched chunk #

2020-08-09 10:54:29,878 : INFO : PROGRESS: pass 4, dispatched chunk #37 = documents up to #76000/363390, outstanding queue size 29
2020-08-09 10:54:29,954 : INFO : PROGRESS: pass 4, dispatched chunk #38 = documents up to #78000/363390, outstanding queue size 30
2020-08-09 10:54:29,964 : INFO : PROGRESS: pass 4, dispatched chunk #39 = documents up to #80000/363390, outstanding queue size 31
2020-08-09 10:54:30,828 : INFO : PROGRESS: pass 4, dispatched chunk #40 = documents up to #82000/363390, outstanding queue size 31
2020-08-09 10:54:31,820 : INFO : PROGRESS: pass 4, dispatched chunk #41 = documents up to #84000/363390, outstanding queue size 31
2020-08-09 10:54:32,784 : INFO : PROGRESS: pass 4, dispatched chunk #42 = documents up to #86000/363390, outstanding queue size 31
2020-08-09 10:54:32,889 : INFO : PROGRESS: pass 4, dispatched chunk #43 = documents up to #88000/363390, outstanding queue size 31
2020-08-09 10:54:33,890 : INFO : PROGRESS: pass 4, dispatched chunk #44 = documents

2020-08-09 10:55:14,933 : INFO : PROGRESS: pass 4, dispatched chunk #86 = documents up to #174000/363390, outstanding queue size 31
2020-08-09 10:55:16,754 : INFO : PROGRESS: pass 4, dispatched chunk #87 = documents up to #176000/363390, outstanding queue size 31
2020-08-09 10:55:17,742 : INFO : PROGRESS: pass 4, dispatched chunk #88 = documents up to #178000/363390, outstanding queue size 31
2020-08-09 10:55:18,760 : INFO : PROGRESS: pass 4, dispatched chunk #89 = documents up to #180000/363390, outstanding queue size 31
2020-08-09 10:55:20,728 : INFO : merging changes from 30000 documents into a model of 363390 documents
2020-08-09 10:55:21,806 : INFO : topic #0 (0.500): 0.001*"deleted" + 0.001*"blah" + 0.000*"blah blah" + 0.000*"duh" + 0.000*"people" + 0.000*"lol" + 0.000*"comment" + 0.000*"link" + 0.000*"enter" + 0.000*"duh duh"
2020-08-09 10:55:22,729 : INFO : topic #1 (0.500): 0.006*"deleted" + 0.005*"people" + 0.005*"just" + 0.005*"like" + 0.005*"dont" + 0.004*"away" + 0.004*"bl

2020-08-09 10:56:03,346 : INFO : PROGRESS: pass 4, dispatched chunk #135 = documents up to #272000/363390, outstanding queue size 31
2020-08-09 10:56:05,434 : INFO : merging changes from 32000 documents into a model of 363390 documents
2020-08-09 10:56:06,486 : INFO : topic #0 (0.500): 0.001*"deleted" + 0.000*"blah" + 0.000*"blah blah" + 0.000*"duh" + 0.000*"lol" + 0.000*"comment" + 0.000*"people" + 0.000*"fap" + 0.000*"enter" + 0.000*"link"
2020-08-09 10:56:06,521 : INFO : topic #1 (0.500): 0.006*"deleted" + 0.005*"people" + 0.005*"just" + 0.005*"like" + 0.005*"dont" + 0.004*"away" + 0.004*"think" + 0.003*"blog" + 0.003*"spam" + 0.003*"im"
2020-08-09 10:56:07,341 : INFO : topic diff=0.043579, rho=0.073187
2020-08-09 10:56:07,414 : INFO : PROGRESS: pass 4, dispatched chunk #136 = documents up to #274000/363390, outstanding queue size 30
2020-08-09 10:56:08,427 : INFO : PROGRESS: pass 4, dispatched chunk #137 = documents up to #276000/363390, outstanding queue size 27
2020-08-09 10:56:0

2020-08-09 10:56:51,384 : INFO : topic #1 (0.500): 0.007*"away" + 0.006*"blog" + 0.006*"spam" + 0.006*"spam blog" + 0.006*"away spam" + 0.006*"blog away" + 0.006*"deleted" + 0.005*"people" + 0.005*"just" + 0.005*"like"
2020-08-09 10:56:51,397 : INFO : topic diff=0.054087, rho=0.073187
2020-08-09 10:57:03,911 : INFO : merging changes from 30000 documents into a model of 363390 documents
2020-08-09 10:57:05,004 : INFO : topic #0 (0.500): 0.001*"blah" + 0.001*"deleted" + 0.001*"blah blah" + 0.000*"lol" + 0.000*"duh" + 0.000*"comment" + 0.000*"fap" + 0.000*"enter" + 0.000*"people" + 0.000*"enter comment"
2020-08-09 10:57:05,031 : INFO : topic #1 (0.500): 0.006*"away" + 0.006*"blog" + 0.006*"spam" + 0.006*"spam blog" + 0.006*"away spam" + 0.006*"blog away" + 0.006*"deleted" + 0.005*"people" + 0.005*"just" + 0.005*"like"
2020-08-09 10:57:05,038 : INFO : topic diff=0.044780, rho=0.073187
2020-08-09 10:57:12,633 : INFO : -11.829 per-word bound, 3638.3 perplexity estimate based on a held-out co

In [18]:
lda.print_topics()

2020-08-09 13:04:37,738 : INFO : topic #0 (0.500): 0.001*"blah" + 0.001*"deleted" + 0.001*"blah blah" + 0.000*"lol" + 0.000*"duh" + 0.000*"comment" + 0.000*"fap" + 0.000*"enter" + 0.000*"enter comment" + 0.000*"upmodded"
2020-08-09 13:04:37,761 : INFO : topic #1 (0.500): 0.006*"away" + 0.006*"deleted" + 0.006*"blog" + 0.006*"spam" + 0.005*"spam blog" + 0.005*"away spam" + 0.005*"blog away" + 0.005*"people" + 0.005*"just" + 0.005*"like"


[(0,
  '0.001*"blah" + 0.001*"deleted" + 0.001*"blah blah" + 0.000*"lol" + 0.000*"duh" + 0.000*"comment" + 0.000*"fap" + 0.000*"enter" + 0.000*"enter comment" + 0.000*"upmodded"'),
 (1,
  '0.006*"away" + 0.006*"deleted" + 0.006*"blog" + 0.006*"spam" + 0.005*"spam blog" + 0.005*"away spam" + 0.005*"blog away" + 0.005*"people" + 0.005*"just" + 0.005*"like"')]

In [19]:
# Transform the docs from the word space to the topic space (like "transform" in sklearn)
lda_corpus = lda[corpus]
lda_corpus

In [20]:
# Store the documents' topic vectors in a list so we can take a peek
lda_docs = [doc for doc in lda_corpus]

In [21]:
# Check out the document vectors in the topic space for the first 5 documents
lda_docs[0:5]

[[(0, 0.030465692), (1, 0.96953434)],
 [(0, 0.25771773), (1, 0.74228233)],
 [(0, 0.14169528), (1, 0.85830474)],
 [(0, 0.30509886), (1, 0.69490117)],
 [(0, 0.21765001), (1, 0.78234994)]]

In [22]:
# Acronynms: Latent Semantic Analysis (LSA) is just another name for 
#  Signular Value Decomposition (SVD) applied to Natural Language Processing (NLP)
lsa = TruncatedSVD(2)
doc_topic = lsa.fit_transform(doc_word)
lsa.explained_variance_ratio_

array([0.89979521, 0.00462399])

In [23]:
topic_word = pd.DataFrame(lsa.components_.round(3),
             index = ["component_1","component_2"],
             columns = count_vectorizer.get_feature_names())
topic_word

ValueError: Shape of passed values is (2, 363390), indices imply (2, 2821736)